# Elasticsearch Installation

## Contents

* Download Elasticsearch public GPG Key and souces list
* Installing Elasticsearch
* Directory layout
* Setting up initial configuration
* Starting Elasticsearch
* Setting security features
* Checking status using teh REST API

### Import Elasticsearch public GPG key into APT

In [ ]:
%%bash
wget -qO - https://artifacts.elastic.co/GPG-KEY-elasticsearch | sudo apt-key add -


#### Validate previous step

##### Validate that the gpg key has been added to your key ring with the following command:

~~~~~
sudo apt-key list
~~~~~

### Add Elastic source List
##### Now, we will add Elastic source list to our system's sources.list.d directory. When making a system update, APT looks at this directory for new sources.

In [ ]:
%%bash
echo "deb https://artifacts.elastic.co/packages/7.x/apt stable main" | sudo tee -a /etc/apt/sources.list.d/elastic-7.x.list

#### Validate previous step

##### We now need to confirm that we have successfully added the new source list into our sources.list.d:

~~~~~
sudo ls /etc/apt/sources.list.d

sudo cat /etc/apt/sources.list.d/elastic-7.x.list 
~~~~~

### Update your system and install elasticsearch

In [ ]:
%%bash
sudo apt update

In [ ]:
%%bash
sudo apt install -y elasticsearch

#### Reload the systemctl daemon
##### After installing each application, it is necesary to reload systemd's darmon using the command below:

~~~~
sudo systemctl daemon-reload
~~~~

### Directory Layout

| Type | Description | Location |
|------|-------------|----------|
| home |  Home of Elasticsearch installation | /usr/share/elasticsearch/ |
| bin  | Binary scripts | /usr/share/elasticsearch/bin |
| conf | Configuration files | /etc/elasticsearch |
| conf | Environment variables (such as heap size) and file descriptors | /etc/default/elasticsearch |
| data | Data files of each index/shard allocated on the node | /var/lib/elasticsearch |
| logs | Log files | /var/log/elasticsearch |
| plugins | Location of plugin files. Each plugin is contain in their own subdirectory | /usr/share/elasticsearch/plugins |

##### You check the contents in some of these directories using:

~~~~~
sudo ls /PATH/TO/DIRECTORY
~~~~~


### Edit elasticsearch configuration

##### Now that we have installed elasticsearch, we will edit Elasticsearch's main configuration file, "elasticsearch.yml". The configuration file is set in YAML format, meaning that indentations and spaces are very important. We will use nano to edit this file:

~~~~~
sudo nano /etc/elasticsearch/elasticsearch.yml
~~~~~

##### Once you access this file lets us take a look at some of the default configuration and change it to our convenience.

##### Let is start by editing the binding address set in the network settings. We will uncomment and change the "network.hosts"

##### From:

~~~~~
#network.host: 192.168.0.1
~~~~~

##### To:

~~~~~
network.host: <ELASTICSEARCH-IP-ADDRESS>
~~~~~

##### We can also observe that the REST API from Elasticsearch listens on port 9200. Any user capable of sending a REST call to this port is able to obtain information from our Elasticsearch cluster (In our case a single node). Due to this, it is better to restrict external traffic to this port using a firewall.

##### We can validate to which IP localhost is associated to by checking the configuration set on our system's hosts file:

~~~~~
sudo cat /etc/hosts
~~~~~

### Start Elasticsearch

##### Now that we have edited our configuration file we can proceed to start elasticsearch

In [ ]:
%%bash
sudo systemctl start elasticsearch
sudo systemctl enable elasticsearch

### Enabling Elasticsearch service

##### Next, let us proceed to enable elasticsearch in order to start up everytime your system boots up. The bootup configutarion file that sets what elasticsearch requires before launching, the home directory, configuration directory, the user under which the process will be launched is set in the file below. This is a file generated by elasticsearch in systemd which controls the boot pro

~~~~~
sudo cat /etc/systemd/system/multi-user.target.wants/elasticsearch.service
~~~~~

### Communicating with Elasticsearch

##### As of now, we are able to communicate with elasticsearch using the REST API. As an example we can verify if it is running by typing the following command:

~~~~~
curl -X GET <ELASTICSEARCH-IP-ADDRESS>:9200
~~~~~


In [ ]:
%%bash
sudo systemctl enable elasticsearch

### Enable Elasticsearch Security Features

##### As we could observe, we can easily send requests using the REST API to elasticsearch on port 9200. This is possible to do from any part of the world is the system is not secured properly. By default, elasticsearch basic and trial licenses have security features disabled by default. 

##### First of all, we will add the following lines to the elasticsearch configuration file to enable the security features:

In [ ]:
%%bash

#sudo sed -i "i88 # --- Security Festures ---" /etc/elasticsearch/elasticsearch.yml
#sudo sed -i "i89 xpack.security.enabled: true" /etc/elasticsearch/elasticsearch.yml

echo "#" | sudo tee --append /etc/elasticsearch/elasticsearch.yml
echo "# --- Security Festures ---" | sudo tee --append /etc/elasticsearch/elasticsearch.yml
echo "xpack.security.enabled: true" | sudo tee --append /etc/elasticsearch/elasticsearch.yml

##### In addition we want to enable single-node discovery as  we are deploying elasticsearch in a single node. In multi-node clusters, we must enable security features ine very single node and it is recommended to configure TLS for internode-communication to prevent other unknown nodes from being added to your cluster.

In [ ]:
%%bash
sudo sed -i "68idiscovery.type: single-node" /etc/elasticsearch/elasticsearch.yml
sudo sed -i "69i#" /etc/elasticsearch/elasticsearch.yml

### Generate passwords for built-in users
##### By default, basic authentication features are enabled after enabling the security features. Before setting up any authentication, we must first make elasticsearch to load the new configuration by restarting it.

In [ ]:
%%bash
sudo systemctl restart elasticsearch

##### Now let we will proceed to generate passwords for built-in users by typing the following command in our terminal:

~~~~~
/usr/share/elasticsearch/bin/elasticsearch-setup-passwords interactive
~~~~~

##### Through this process you will set a password for the following built-in users: elastic, apm_system, kibana, logstash_system, beats_system, and remote_monitoring_user.

| User | Description |
|------|-------------|
| elastic |A built-in superuser. See Built-in roles. |
| kibana | The user Kibana uses to connect and communicate with Elasticsearch. |
| logstash_system | The user Logstash uses when storing monitoring information in Elasticsearch. |
| beats_system | The user the Beats use when storing monitoring information in Elasticsearch. |
| apm_system | The user the APM server uses when storing monitoring information in Elasticsearch. |
| remote_monitoring_user |The user Metricbeat uses when collecting and storing monitoring information in Elasticsearch. It has the remote_monitoring_agent and remote_monitoring_collector built-in roles. |

##### Alternativery, you can type the command below to generate a random passowrd for each user using the command below:

~~~~
/usr/share/elasticsearch/bin/elasticsearch-setup-passwords auto
~~~~


##### Now that we have set our authentication in elasticsearch, let us try again to connect to elasticsearch using the REST API.

##### Type in your terminal the following:

~~~~~
curl -X GET <ELASTICSEARCH-IP-ADDRESS>:9200
~~~~~

##### What results did you got?

##### Now let us try using our new credentials using the elastic SUPERUSER

~~~~
curl -u elastic:elasticsiem -XGET <ELASTICSEARCH-IP-ADDRESS>:9200
~~~~

##### By default elsaticsearch uses the internal native real to manage user's authentication.

##### For more information about the built-in roles, authentication realms, and to set passwords for a specific users please see the following links.

### References

https://www.elastic.co/guide/en/elastic-stack-overview/current/built-in-roles.html

https://www.elastic.co/guide/en/elastic-stack-overview/current/realms.html

https://www.elastic.co/guide/en/elastic-stack-overview/current/built-in-users.html

https://www.elastic.co/guide/en/beats/filebeat/current/directory-layout.html